
**1. 필요한 라이브러리 설치**
1. Python: >=3.6
2. Tensorflow: 1.12.0
3. smac: 0.8.0

In [ ]:
#//3. SMAC 설치
!pip install smac==0.8.0

In [ ]:
#//텐서플로우 1.12 설치를 위한 콘다코랩 설치
!pip install -q condacolab
import condacolab
condacolab.install()

!conda activate tf_env
!conda install -c conda-forge tensorflow=1.12.0

In [ ]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [ ]:
import sys
import tensorflow as tf
import smac

# Python 버전 확인
print("Python 버전:", sys.version)

# 텐서플로우 버전 확인
print("텐서플로우 버전:", tf.__version__)

# SMAC 버전 확인
print("SMAC 버전:", smac.__version__)

**2. LSTM_MSNet 코드 다운로드(GitHub)**

In [ ]:
#// Git 저장소에서 소스 코드를 복제(clone)하는 명령어
# (프로젝트의 전체 소스 코드를 가져와 현재 작업 중인 디렉토리에 복제)
!git clone https://github.com/kasungayan/LSTMMSNet

Cloning into 'LSTMMSNet'...
remote: Enumerating objects: 222, done.
remote: Counting objects: 100% (222/222), done.
remote: Compressing objects: 100% (167/167), done.
remote: Total 222 (delta 77), reused 122 (delta 44), pack-reused 0
Receiving objects: 100% (222/222), 5.49 MiB | 7.98 MiB/s, done.
Resolving deltas: 100% (77/77), done.


In [ ]:
#//작업 디렉토리가 content/LSTMMSNet으로 변경(파일 및 폴더에 접근하고 작업 가능)
%cd /content/LSTMMSNet

/content/LSTMMSNet


In [ ]:
import os

# 경로 설정
project_root = "/absolute/path/to/project/root"
cocob_optimizer = "/absolute/path/to/src/LSTM-Models/external_packages/cocob_optimizer"

# PYTHONPATH 설정
os.environ["PYTHONPATH"] = f"{os.environ.get('PYTHONPATH', '')}:{project_root}:{cocob_optimizer}"


1단계: 전처리 레이어
- 정규화 및 분산 안정화를 수행
- 계절성 분해를 수행

In [ ]:
%cd /content/LSTMMSNet/src/LSTM-Models/preprocess_scripts/DS/MSTL-DS/
!python energy_create_tfrecords_mean_hourly_mstl.py

3. 데이터 전처리
*   학습, 검증 및 테스트 스크립트 생성
*   TRecord 생성(STM-MSNet을 교육할 때 Tensorflow API에서 제공하는 tfrecords 기능을 사용하여 모델을 더 빠르게 실행)



⏬이 코드는 TFRecord 파일로 데이터를 저장하기 위해 필요한 데이터를 읽고 변환하는 기능을 제공(데이터셋 ->TFRecord 형식)

In [ ]:
%cd /content/LSTMMSNet/src/LSTM-Models/tfrecords_handler/moving_window/
!python energy_tfrecord_DS_writer.py

⏬ TFRecordReader 클래스를 사용하여 TFRecord 파일에서 데이터를 읽어오고, 파싱하여 필요한 데이터를 추출

In [ ]:
%cd /content/LSTMMSNet/src/LSTM-Models/tfrecords_handler/moving_window/
!python energy_tfrecord_DS_reader.py

2단계: 순환 레이어
- LSTM 기반 쌓기 아키텍처(여러 개의 LSTM 레이어를 쌓아서 네트워크를 구성)

- 기본 모델
- 스테킹 모델

generic_model_energy_DS_trainer
1. 필요한 라이브러리 및 모듈 임포트
2. 하이퍼파라미터 설정:SMAC 알고리즘을 통해 최적화할 하이퍼파라미터의 범위를 지정
3. 모델 아키텍처 및 옵티마이저 설정: 다양한 모델 아키텍처(스태킹 모델, 시퀀스 투 시퀀스 모델, 어텐션 모델 등)과 옵티마이저(Adagrad, Adam, COCOB 등)를 설정
4. 학습 함수 설정: SMAC 알고리즘의 타겟 알고리즘으로 사용될 학습 함수를 정의
5. SMAC 알고리즘 설정: SMAC의 구성 요소인 Configuration Space와 Scenario를 설정하고, SMAC 객체를 생성하여 최적화를 수행
6. 주요 함수 실행: main 함수에서 위의 함수들을 호출하여 모델 학습과 최적화를 실행
⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬


In [ ]:
%cd /content/LSTMMSNet/src/LSTM-Models/
!python generic_model_energy_DS_trainer.py

/content/LSTMMSNet/src/LSTM-Models
2023-06-02 09:02:28.765180: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/LSTMMSNet/src/LSTM-Models/generic_model_energy_DS_trainer.py", line 5, in <module>
    from generic_mean_m4hourly_test import testing
ModuleNotFoundError: No module named 'generic_mean_m4hourly_test'


이 모델은 주어진 시계열 데이터를 기반으로 예측 모델을 학습하고, 학습된 모델을 평가하는 기능을 제공

1. 클래스 초기화: StackingModelTrainer 클래스는 여러 매개변수를 받아 객체를 초기화합니다. 이 매개변수들은 모델의 구성과 학습에 필요한 설정 값을 제공합니다.

2. 손실 함수 정의: __l1_loss 및 __l2_loss 메서드는 예측 값과 실제 값 사이의 손실을 계산하는 함수입니다. L1 손실과 L2 손실을 계산하여 반환합니다.

3. 모델 학습: train_model 메서드는 주어진 하이퍼파라미터와 데이터를 사용하여 스태킹 모델을 학습합니다. 옵티마이저와 레이어 수, 셀 크기 등의 하이퍼파라미터를 설정하고, 입력과 출력 데이터를 placeholder로 정의합니다. 그 후 RNN 셀과 레이어를 구성하고, 학습 및 추론에 사용되는 변수를 설정합니다. 학습 데이터와 검증 데이터셋을 tf.data.Dataset으로 준비하고, 학습과 평가를 반복하여 수행합니다.

4. 학습 및 검증 데이터 처리: 학습 데이터와 검증 데이터는 TFRecord 형식으로 제공됩니다. TFRecordReader를 사용하여 데이터를 읽고, 패딩 처리 및 배치 처리를 수행합니다.

5. 최적화 및 평가: 학습 데이터를 사용하여 모델을 학습하고, 검증 데이터를 사용하여 학습된 모델을 평가합니다. 검증 데이터에 대한 예측 결과를 계산하고, SMAPE(Symmetric Mean Absolute Percentage Error) 값을 계산하여 모델의 성능을 평가합니다.
⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬


In [ ]:
%cd /content/LSTMMSNet/src/LSTM-Models/rnn_architectures/stacking_model/
!python energy_stacking_model_DS_trainer.py

/content/LSTMMSNet/src/LSTM-Models/rnn_architectures/stacking_model
  File "/content/LSTMMSNet/src/LSTM-Models/rnn_architectures/stacking_model/energy_stacking_model_DS_trainer.py", line 212
    converted_validation_output = converted_validation_output - 1
TabError: inconsistent use of tabs and spaces in indentation


In [ ]:
%cd /content/LSTMMSNet/src/LSTM-Models/
!python generic_model_energy_DS_test.py

/content/LSTMMSNet/src/LSTM-Models
2023-06-02 09:08:03.888125: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/LSTMMSNet/src/LSTM-Models/generic_model_energy_DS_test.py", line 7, in <module>
    from rnn_architectures.stacking_model.stacking_mean_model_tester import StackingModelTester as StackingModelTester
ModuleNotFoundError: No module named 'rnn_architectures.stacking_model.stacking_mean_model_tester'


**스태킹 모델(Stacking Model)의 테스트 코드 구성**
1. 필요한 패키지 및 모듈 가져오기
2. StackingModelTester 클래스 정의
3. TensorFlow 그래프 설정
4. 손실 함수와 최적화 기법 설정
5. 데이터 준비
6. TensorFlow 세션 실행
7. 예측값 반환
⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬⏬



In [ ]:
%cd /content/LSTMMSNet/src/LSTM-Models/rnn_architectures/stacking_model/
!python energy_stacking_model_DS_tester.py

/content/LSTMMSNet/src/LSTM-Models/rnn_architectures/stacking_model
2023-06-02 09:26:58.698179: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/LSTMMSNet/src/LSTM-Models/rnn_architectures/stacking_model/energy_stacking_model_DS_tester.py", line 3, in <module>
    from tfrecords_handler.moving_window.tfrecord_mean_reader import TFRecordReader
ModuleNotFoundError: No module named 'tfrecords_handler'
